## Градиент стратегии: REINFORCE.

Теорема о градиенте стратегии связывает градиент целевой функции  и градиент самой стратегии:

$$\nabla_\theta J(\theta) = \mathbb{E}_\pi [\nabla_\theta \ln \pi_\theta(a \vert s) Q^\pi(s, a)]$$

Если использовать метод Монте-Карло в качестве несмещенной оценки $Q^\pi(s, a)$ отдачу $R_t$, то тогда происходит переход к алгоритму REINFORCE и обновление весов будет осуществляться по правилу:

$$\nabla_\theta J(\theta) = [R_t \ln \nabla_\theta \pi_\theta(A_t \vert S_t)]$$

In [ ]:
try:
    import colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip install "gymnasium[classic-control, atari, accept-rom-license]" --quiet
    !pip install piglet --quiet
    !pip install imageio_ffmpeg --quiet
    !pip install moviepy==1.0.3 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 54.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.5/67.5 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 KB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
import torch.nn as nn
from torch.distributions import Categorical
import gymnasium as gym
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

### Основной цикл

In [ ]:
def print_mean_reward(step, episode_rewards):
    if not episode_rewards:
        return

    mean_reward = round(sum(episode_rewards) / len(episode_rewards), 2)
    print(f"step: {str(step).zfill(6)}, mean reward: {mean_reward}")
    return mean_reward
    
    
class Rollout:
    def __init__(self):
        self.logprobs = []
        self.actions = []
        self.rewards = []
        self.is_terminals = []
        
    def append(self, log_prob, reward, done):
        self.logprobs.append(log_prob)
        self.rewards.append(reward)
        self.is_terminals.append(done)

In [ ]:
# Реализуйте класс, задающий стратегию агента.
# Подсказки:
#     1) можно воспользоваться базовым классом `torch.nn.Module`,
#     2) размер нейронной сети можно выбрать таким: (input_dim, hidden_dim, output_dim),
#     3) в качестве функции активации возьмите гиперболический тангенс или ReLU
#     4) подумайте, как получить на выходе из нейронной сети вероятности действий,
#     5) для выбора действия в соответствии со стратегией, можно воспользоваться `torch.distributions.Categorical`
#     6) помните, что помимо самого действия вам позже также пригодится логарифм его вероятности
####### Здесь ваш код ########
class Agent(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_hidden_layers):
        super().__init__()
        modules = [nn.Linear(input_dim, hidden_dim), nn.Tanh()]
        for _ in range(n_hidden_layers):
            modules.extend([nn.Linear(hidden_dim, hidden_dim), nn.Tanh()])
        modules.extend([nn.Linear(hidden_dim, output_dim), nn.Softmax(-1)])
        self.net = nn.Sequential(*modules)
    def forward(self, state_x: torch.Tensor):
        probs = self.net(state_x)
        distr = torch.distributions.Categorical(probs=probs)   
        action = distr.sample()
        log_prob = distr.log_prob(action)
        return action.item(), log_prob[None]
##############################


class ReinforceAgent:
    def __init__(self, state_dim, hidden_dim, action_dim, n_latent_var, lr, gamma):
        self.lr = lr
        self.gamma = gamma

        # Инициализируйте стратегию агента и SGD оптимизатор (например, `torch.optim.Adam)`)
        ####### Здесь ваш код ########
        self.policy = Agent(state_dim, hidden_dim, action_dim, n_latent_var).to(device)
        print(self.policy)
        self.optimizer = torch.optim.Adam(params=self.policy.parameters(), lr = lr)
        ##############################
        self.action_dim = action_dim
        
    def act(self, state):
        # Произведите выбор действия и верните кортеж (действие, логарифм вероятности этого действия)
        ####### Здесь ваш код ########
        if isinstance(state, np.ndarray):
            state = torch.tensor(state, device=device)
        return self.policy(state)
        ##############################

    def update(self, rollout: Rollout):
        # Конвертируйте накопленный список вознаграждений в список отдач. Назовем его `rewards`
        # Подсказки:
        #    1) обход списка стоит делать в обратном порядке, 
        #    2) не забывайте сбрасывать отдачу при окончании эпизода
        # rewards = 
        ####### Здесь ваш код ########
        rewards = rollout.rewards
        
        cumulative_rewards = [rewards[-1]]
        for i in range(len(rewards)-2,-1,-1):
            cumulative_rewards.append(rewards[i] + self.gamma * cumulative_rewards[-1] * rollout.is_terminals[i])
           
        cumulative_rewards = torch.tensor(cumulative_rewards[::-1], device = device)[:, None] 

        cumulative_rewards = (cumulative_rewards - torch.mean(cumulative_rewards)) / (torch.std(cumulative_rewards) + 1e-6)
        ##############################
        # Вычислите ошибку `loss` и произведите шаг обновления градиентным спуском
        # Подсказки: используйте `.to(device)`, чтобы разместить тензор на соотв. цпу/гпу
        ####### Здесь ваш код ########
        self.optimizer.zero_grad()

        loss = -torch.mean(torch.stack(rollout.logprobs).to(device) * cumulative_rewards)
        # print(loss.item())       
        loss.backward()
        self.optimizer.step()

        ##############################


def run(env: gym.Env, hidden_size: int, n_layers: int, lr: float, gamma: float, max_episodes: int, rollout_size: int):
    ####### Здесь ваш код ########
    ##############################
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    print(action_dim)

    agent = ReinforceAgent(state_dim, hidden_size, action_dim, n_layers, lr, gamma)
    step = 0
    rollout = Rollout()
    episode_rewards = []

    for i_episode in range(1, max_episodes + 1):
        cumulative_reward = 0        
        terminated = False
        state, _ = env.reset()
        
        while not terminated:
            step += 1
            
            action, log_prob = agent.act(state)
            state, reward, terminated, truncated, _ = env.step(action)

            rollout.append(log_prob, reward, not terminated)
            cumulative_reward += reward
            terminated |= truncated

        episode_rewards.append(cumulative_reward)

        
        # выполняем обновление
        if len(rollout.rewards) >= rollout_size:
            agent.update(rollout)
            mean_reward = print_mean_reward(step, episode_rewards) 
            if mean_reward >= 200:
                print('Принято!')
                return
            rollout = Rollout()
            episode_rewards = []

### Определяем гиперпараметры и запускаем обучение

In [ ]:
from gymnasium.wrappers.time_limit import TimeLimit
env_name = "CartPole-v1"

run(
    env = TimeLimit(gym.make(env_name), 1000),
    max_episodes = 50000,  # количество эпизодов обучения
    hidden_size = 64,  # кол-во переменных в скрытых слоях
    n_layers=1,
    rollout_size = 500,  # через столько шагов стратегия будет обновляться
    lr = 0.01, # learning rate
    gamma = 0.995,  # дисконтирующий множитель,
)

2
Agent(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=2, bias=True)
    (5): Softmax(dim=-1)
  )
)
step: 000505, mean reward: 22.95
step: 001028, mean reward: 32.69
step: 001556, mean reward: 58.67
step: 002139, mean reward: 83.29
step: 002798, mean reward: 131.8
step: 003489, mean reward: 172.75
step: 004096, mean reward: 121.4
step: 004632, mean reward: 107.2
step: 005186, mean reward: 184.67
step: 005757, mean reward: 142.75
step: 006301, mean reward: 108.8
step: 006876, mean reward: 82.14
step: 007421, mean reward: 77.86
step: 007945, mean reward: 87.33
step: 008530, mean reward: 97.5
step: 009071, mean reward: 135.25
step: 009679, mean reward: 121.6
step: 010319, mean reward: 160.0
step: 010912, mean reward: 148.25
step: 011428, mean reward: 129.0
step: 012059, mean reward: 157.75
step: 012770, mean reward: 237.